In [54]:
# TODO
# Ask Sarah about how editions are separated/duplicates. then, code this up for all editions

# run two_data & port re-format & visualisation from CW2


# check that clean_up_definitions does a proper clean inplace
# look through CW2 cleanup & code: what could be ported over
# for_later notes eg. \n clean

# DONE
# words/characters/volumes per edition

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

In [2]:
# reading in index file

inventory = pd.read_csv("encyclopaediaBritannica-inventory.csv", header=None)
inventory.columns = ['file','volume']
# print(inventory)
print("\nNumber of text files: " + str(len(inventory)))


Number of text files: 195


In [3]:
first_ed = inventory[:3]
list(first_ed['volume']) # list() to print whole text

['Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan … - First edition, 1771, Volume 1, A-B - EB.1',
 'Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan … - First edition, 1771, Volume 2, C-L - EB.1',
 'Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan … - First edition, 1771, Volume 3, M-Z - EB.1']

In [21]:
second_ed = inventory[6:16]
list(second_ed['volume']) # list() to print whole text

['Encyclopaedia Britannica - Second edition, Volume 1, A-AST - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 2, Astronomy-BZO - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 3, C - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 4, D-F - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 5, G-J - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 6, K-Medicine - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 7, Medicines-Optics - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 8, Optics-Poetry - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 9, POI-SCU - EB.4',
 'Encyclopaedia Britannica - Second edition, Volume 10, SCU-Appendix - EB.4']

In [31]:
two_editions = [first_ed, second_ed]

In [45]:
# Reading in text files and generate volume, word and length counts

def basic_data(editions):
    
    data = []
    
    for ed in editions: 
        length = 0
        words = 0
        volumes = 0

        for index, row in ed.iterrows():
            print("Reading: " + row['file'])

            f = open('text/' + row['file'], 'r', encoding="utf8")
            content = f.read()
            length += len(content)
            volumes += 1

            words += len(content.split())

            f.close()
            
        data.append([volumes, length, words])
        data_df=pd.DataFrame(data,columns=['volumes', 'length_chars','length_words'])
    return data_df

In [46]:
basic_data(two_editions)

Reading: 144133901.txt
Reading: 144133902.txt
Reading: 144133903.txt
Reading: 144850370.txt
Reading: 144850373.txt
Reading: 144850374.txt
Reading: 144850375.txt
Reading: 144850376.txt
Reading: 144850377.txt
Reading: 144850378.txt
Reading: 144850379.txt
Reading: 190273289.txt
Reading: 190273290.txt


,volumes,length_chars,length_words
0,3,14223433,2567769
1,10,56865594,10173856


## Most referenced topics

### Helper functions

#### For cleanup

In [52]:
def replace_by(s, a, b):
    """ perform a regex replacement, prints number of occurrences found and returns a string.
    s: string to make changes in
    a: string to remove 
    b: string to add
    returns: a string"""
    
    # print("Replacing \"" + a + "\" by \"" + b + "\", found " + str(len(re.findall(a, s))) + "...")
    new = re.sub(a, b, s)
    
    return new

In [53]:
# Using regex cleanup ideas from: https://sites.temple.edu/tudsc/2014/08/12/text-scrubbing-hacks-cleaning-your-ocred-text/

# Further resources: https://programminghistorian.org/en/lessons/cleaning-ocrd-text-with-regular-expressions
# https://datascience.stackexchange.com/questions/20536/how-to-improve-ocr-scanning-results

def clean_up(s):
    """ Does a minimal cleanup of a string of text
    returns: a string
    """
    # print("Initial length: " + str(len(s)))
    s2 = replace_by(s, 'tbe', 'the')
    s3 = replace_by(s2, 'tiie', 'the')
    s4 = replace_by(s3, 'liis', 'his')
    s5 = replace_by(s4, 'bis', 'his')
    s6 = replace_by(s5, '■', '')
    s7 = replace_by(s6, 'ib','in') # was noticed by Ava in .txt files
    s8 = replace_by(s7, '¬','')
    s9 = replace_by(s8, '.(\.\.+)', '') # multiple periods; what about ellipsis though?
    # print("Clean up done!")
    return s9 # CAREFUL to always pass the right one to next, and return

In [63]:
# This uses the list of header/definition/refs dataframes for each edition
# Check that this doesn't just do a reference replace, actually changes editions that are passed to it

def clean_up_definitions(editions):
    editions_cleaned = []
    for ed in editions:
        # realistically, a definition can't be shorter than 5 letters, even if it is simply a redirection of the form of "See x"
        ed = ed.loc[ed['def_length'] > 5]
        # remove headers with two letters as they are probably mostly noise
        ed = ed.loc[ed['header_length'] > 2]
        # remove headers longer than 40 characters, as these are likely to be noise
        ed = ed.loc[ed['header_length'] < 40]
        
        # We might also want to drop header duplicates, as each entry is only defined once
        # A good portion of these duplicates are probably noise (such as annotations in roman numerals)
        # We're not selecting the correct definition out of the duplicates here, just dropping them for the sake of simplicity
        ed.drop_duplicates(subset ="headers", keep = False, inplace = True)
        
        editions_cleaned.append(ed)
        
    return editions_cleaned

#### For extracting words and their definitions, and references ("See x")

In [56]:
# Uses RegEx to identify the definition of a word as being the content between the uppercase word and the next uppercase word

def find_definition(word, volume):
    start = re.search(word, volume).start()
    start_next = re.search(word, volume).end()
    
    second_word = re.search("[A-Z][A-Z]+", volume[start_next:])
    
    if second_word: # checking that second capitalised word exists (prevents NoneType exception)
        end = re.search("[A-Z][A-Z]+", volume[start_next:]).start()
        return volume[start_next+2:start_next+end] # +2 to ignore comma and space before a definition
    else:
        return ""

In [49]:
# Extracts entries (headers), their definitions, their respective lengths, and all references ("See x") from an edition

def extract_info(edition):
    
    headers = []
    definitions = []
    refs = [] # for counting up all instances of "See x" in volumes  
    
    for index, row in edition.iterrows():
        current_headers = []
        
        print("Reading: " + row['file'])
        f = open('text/' + row['file'], 'r', encoding="utf8")
        content = f.read()
        content = clean_up(content) # check cleanup does something
        # using https://stackoverflow.com/questions/9525993/get-consecutive-capitalized-words-using-regex
        # note: this ReGex is not entirely correct, as the first will select "MARTIAL" and "LAW"
        # "MARTIAL" will be cleaned out later due to having an empty definition, but "LAW" will have duplicates
        
        current_headers = re.findall('[A-Z][A-Z]+', content) # at least two uppercase letters following each other
        current_headers += re.findall('([A-Z][A-Z]+(?=\s[A-Z])(?:\s[A-Z][A-Z]+)+)', content) # two or more uppercase words
        refs += re.findall('See [^,\.]*', content) # matches "See x" until a comma or a period
        
        for word in current_headers:
            definitions.append(find_definition(word, content))
            
        headers = headers + current_headers
        
        f.close()

    data = pd.DataFrame(headers, columns =['headers'])
    data['definition'] = definitions
    data['header_length']  = data['headers'].str.len()
    data['def_length']  = data['definition'].str.len()
    return data, refs

In [50]:
# Generates a list of dataframes with words and their definitions for each edition

def words_and_definitions(editions):
    
    all_edition_data = []
    all_references = []
    
    for ed in editions:
        edition_data, references = extract_info(ed)
        all_edition_data.append(edition_data)
        all_references.append(references)
    return all_edition_data, all_references

In [55]:
two_data, two_refs = words_and_definitions(two_editions)

Reading: 144133901.txt
Reading: 144133902.txt
Reading: 144133903.txt
Reading: 144850370.txt
Reading: 144850373.txt


AttributeError: 'NoneType' object has no attribute 'start'

In [57]:
second_data, refs = extract_info(second_ed)

Reading: 144850370.txt
Reading: 144850373.txt
Reading: 144850374.txt
Reading: 144850375.txt
Reading: 144850376.txt
Reading: 144850377.txt
Reading: 144850378.txt
Reading: 144850379.txt
Reading: 190273289.txt
Reading: 190273290.txt


In [64]:
second_data_cleaned = clean_up_definitions([second_data])

In [69]:
second_data_cleaned[0]

,headers,definition,header_length,def_length
111,AACH,"a little town in Germany, in the circle of\nSu...",4,309
112,AAHUS,"a little town in Germany, in the circle of\nWe...",5,209
115,AARON,"high-prieft of the Jews, and brother to\nMofes...",5,857
118,AARSENS,"Peter), a painter, called in Italy Pietro\nEon...",7,402
122,ABACATUAIA,"in ichthyology, a barbarous\nname of the zeus ...",10,62
...,...,...,...,...
58499,WAR\nWAR,9184 ]\nw.\nSee Virgula Divina in this\n,7,38
58500,AX AX,) uinquefolium.\n,5,16
58501,WIN TERANTA,romatic a. Tlate,11,17
58504,CCC IV,)\ncccv. C\ncccvio\n,6,17
